https://huggingface.co/blog/fine-tune-whisper

In [1]:
whisper_model = "openai/whisper-small"#"openai/whisper-medium"#"openai/whisper-large-v3"
lang_id = "af"
model_dir="./afr_FLEURS_model"
output_dir="./afr_FLEURS_model_output"

In [4]:
from datasets import load_from_disk, Audio, load_dataset
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

In [5]:
dataset_dict = load_dataset("google/fleurs", "af_za")
print(dataset_dict)

C:\Users\Augustijn\anaconda3\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 1032
    })
    validation: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 198
    })
    test: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 264
    })
})


In [6]:
dataset_dict['train'][0]

{'id': 766,
 'num_samples': 72960,
 'path': 'C:\\Users\\Augustijn\\.cache\\huggingface\\datasets\\downloads\\extracted\\d23995f4a224aa5de9a8afb10ca2a4f6cf9c3b82b4a0de28edbdb962f933af80\\10010386886416577453.wav',
 'audio': {'path': 'train/10010386886416577453.wav',
  'array': array([0.        , 0.        , 0.        , ..., 0.00410163, 0.0040049 ,
         0.00438273]),
  'sampling_rate': 16000},
 'transcription': "families met kinders is nie 'n algemene verskynsel nie maar party koshuise laat hulle in privaat kamers toe",
 'raw_transcription': 'Families met kinders is nie ‘n algemene verskynsel nie, maar party koshuise laat hulle in privaat kamers toe.',
 'gender': 1,
 'lang_id': 0,
 'language': 'Afrikaans',
 'lang_group_id': 3}

In [11]:
for entry in dataset_dict['train']:
    if(entry["audio"]["sampling_rate"] == 16000):
        pass
    else:
        print("needs resampling")

In [12]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(whisper_model)

tokenizer = WhisperTokenizer.from_pretrained(whisper_model, language=lang_id, task="transcribe")

processor = WhisperProcessor.from_pretrained(whisper_model, language=lang_id, task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
#Skip this step as sampling rate is already correct for FLEURS
#dataset_dict = dataset_dict.cast_column("file", Audio(sampling_rate=16000))

In [14]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

dataset_dict = dataset_dict.map(prepare_dataset, remove_columns=dataset_dict.column_names["train"])

Map:   0%|          | 0/1032 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/264 [00:00<?, ? examples/s]

In [15]:
model = WhisperForConditionalGeneration.from_pretrained(whisper_model)
'''
#Freeze the first 16 layers of the model to reduce computational load.
for param in model.model.encoder.layers[:16].parameters():
    param.requires_grad = False
''' 
model.generation_config.language = lang_id
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None    

In [16]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [17]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [18]:
metric = evaluate.load("wer")


In [19]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,  
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False  # Set this according to your evaluation metric
)

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)

C:\Users\Augustijn\anaconda3\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
trainer.train()

C:\Users\Augustijn\AppData\Roaming\Python\Python310\site-packages\transformers\models\whisper\modeling_whisper.py:697: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,Wer
1,No log,0.830235,36.606292
2,No log,0.786496,34.270734
3,No log,0.794380,40.204957
4,No log,0.816643,37.797903


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfi

TrainOutput(global_step=260, training_loss=0.33458199134239786, metrics={'train_runtime': 5861.182, 'train_samples_per_second': 1.761, 'train_steps_per_second': 0.111, 'total_flos': 1.19128053252096e+18, 'train_loss': 0.33458199134239786, 'epoch': 4.0})

In [22]:
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


('./afr_FLEURS_model\\tokenizer_config.json',
 './afr_FLEURS_model\\special_tokens_map.json',
 './afr_FLEURS_model\\vocab.json',
 './afr_FLEURS_model\\merges.txt',
 './afr_FLEURS_model\\normalizer.json',
 './afr_FLEURS_model\\added_tokens.json')

In [23]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.7864958047866821, 'eval_wer': 34.27073403241182, 'eval_runtime': 724.0048, 'eval_samples_per_second': 0.273, 'eval_steps_per_second': 0.018, 'epoch': 4.0}


In [24]:
test_results = trainer.evaluate(eval_dataset=dataset_dict["test"])
print(test_results)

{'eval_loss': 0.7801522016525269, 'eval_wer': 31.93073593073593, 'eval_runtime': 984.2207, 'eval_samples_per_second': 0.268, 'eval_steps_per_second': 0.017, 'epoch': 4.0}
